# V1.0 RAG with Custom Prompt
This version focus on RAG with custom prompt.\
Custom prompt is designed to invoke code generation by the LLM.

In [1]:
%pip install --user -qU langchain
%pip install --user -qU langchain_community
%pip install --user -qU langchain_chroma
%pip install --user -qU langchain-openai
%pip install --user -qU langchainhub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [4]:
folder_path = "ca-md"

documents = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        loader = UnstructuredMarkdownLoader(file_path)
        loaded_docs = loader.load()
        # Add source metadata to each document
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        documents.extend(loaded_docs)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True
)
text_chunks = text_splitter.split_documents(documents)

In [6]:
vectorstore = Chroma.from_documents(documents=text_chunks, embedding=OpenAIEmbeddings())

In [7]:
retriever = vectorstore.as_retriever()

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

## Prompt 1
Idea: Using system prompt, instruct the LLM to use the documentation to help answer the user's request.

In [15]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [16]:
response = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response["answer"])

To create a strategy based on the Relative Strength Index (RSI) for trading Bitcoin, you can use the RSI indicator logic you provided. Here's a simple Python script using the `pandas` library to demonstrate how you can implement this strategy:

```python
import pandas as pd

# Sample RSI values for Bitcoin
rsi_values = [35, 25, 28, 75, 80, 68, 20, 40, 60]

# Create a DataFrame with the RSI values
df = pd.DataFrame(rsi_values, columns=['RSI'])

# Define the RSI thresholds for buying and selling
buy_threshold = 30
sell_threshold = 70

# Initialize a variable to hold the position status (0 - not in position, 1 - in buy position, 2 - in sell position)
position_status = 0

# Create a loop to iterate over the RSI values and implement the strategy
for index, row in df.iterrows():
    rsi = row['RSI']
    
    if rsi < buy_threshold and position_status != 1:
        print(f"RSI is {rsi}. Buy Bitcoin.")
        position_status = 1
    elif rsi > sell_threshold and position_status != 2:
        

In [17]:
for document in response["context"]:
    print(f"Source: {document.metadata['source']}")
    #print('#############################################################')

Source: rsi.md
Source: rsi.md
Source: 01-sync-by-position.md
Source: dmi.md


LLM is able to find the correct documentation of the technical indicator (in this case RSI). \
However, the retriever is only able to retrieve documentation for the technical indicator and not the necessary markdown documentations for generating strategy code (hello-world.mdx).

## Prompt 2
Idea: under system prompt, explicitly tells the LLM to generate code that follows the template found in 'hello-world.mdx'

Issue: the retriever only uses the input query to retrieve relevant documents. That is, it does not look at the system prompt for identifying what documents to retrieve.

In [12]:
system_prompt = (
    """
    You are a coding assistant with expertise in Crypto Arsenal's documentation.
    You are task with generating code based on the template in file 'hello-world.mdx' that help executes the user's trading sytrategy.
    Here is a set of Crypto Arsenal's documentation retrieved to answer the question: \n -------- \n {context} \n -------- \n Ensure any code you provide can be executed with all the required imports and variables defined. If you do not know the answer or require further clarification, just say that you do not know.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [13]:
response = rag_chain.invoke({"input": "I want to create a strategy where I buy Bitcoin if its RSI drops below 30 and sell if it goes above 70."})
print(response["answer"])

To create a trading strategy based on the RSI (Relative Strength Index) for buying Bitcoin when its RSI drops below 30 and selling when it goes above 70, you can use the following Python code snippet along with the `ta` library for calculating the RSI. Make sure to have the `ta` library installed by running `pip install ta`.

Here's a sample code snippet for your strategy:

```python
# Import the necessary libraries
import pandas as pd
import ta

# Load your Bitcoin price data into a pandas DataFrame (replace this with your actual data)
# Make sure your DataFrame has columns named 'High', 'Low', and 'Close'
data = pd.read_csv('bitcoin_price_data.csv')

# Calculate the RSI with a period of 14 days using the ta library
data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

# Implement your trading strategy based on RSI
# Initialize a variable to track if we are currently in a position
in_position = False

# Loop through the RSI values
for index, row in data.iterrows():
 

In [14]:
for document in response["context"]:
    print(f"Source: {document.metadata['source']}")
    #print('#############################################################')

Source: rsi.md
Source: rsi.md
Source: 01-sync-by-position.md
Source: dmi.md
